In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [ ]:
df=pd.read_csv('Alodokter - Alodokter.csv')
df.isnull().sum()

Curhat                 0
Judul                  0
Link                   0
Topic                  0
Keluarga             136
Percintaan           136
Anak Remaja          136
Pengembangan Diri    135
Trauma               137
Phobia               137
Masalah Emosi        135
Bullying             136
Bukan Psikologi      137
dtype: int64

In [ ]:
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)

In [ ]:
df.isnull().sum()

Curhat               0
Judul                0
Link                 0
Topic                0
Keluarga             0
Percintaan           0
Anak Remaja          0
Pengembangan Diri    0
Trauma               0
Phobia               0
Masalah Emosi        0
Bullying             0
Bukan Psikologi      0
dtype: int64

In [ ]:
import ast
file = open("combined_slang_words.txt")
content = file.read()
dicti = ast.literal_eval(content)
slang = pd.DataFrame(dicti.items(),columns=['original','replacement'])
slang.head()

,original,replacement
0,pdahal,padahal
1,kluar,keluar
2,hbis,habis
3,tnya,tanya
4,prnah,pernah


In [ ]:
stopwords = pd.read_csv('combined_stop_words.txt',sep='\n',header=None)
stopwords.columns = ['stopword']
stopwords.head()

,stopword
0,adalah
1,adapun
2,agaknya
3,akan
4,akhir


In [ ]:
def lowercase(text):
    return text.lower()

def remove_repeated_char(text):
    return ' '.join([re.sub(r'(.+?)\1+$', r'\1\1',word) for word in text.split(' ')])

def remove_unnecessary_char(text):
    text = re.sub('\n',' ',text) # Remove every '\n'
    text = re.sub('\r',' ',text) # Remove every '\r'
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))',' ',text) # Remove every URL
    text = re.sub('  +', ' ', text) # Remove extra spaces
    return text

def remove_nonaplhanumeric(text):
    text = re.sub('[^a-zA-Z0-9]+', ' ', text) 
    return text

slang_dict_map = dict(zip(slang['original'], slang['replacement']))
def formalize_words(text):
    return ' '.join([slang_dict_map[word] if word in slang_dict_map else word for word in text.split(' ')])

def remove_stopword(text):
    text = ' '.join(['' if word in stopwords.stopword.values else word for word in text.split(' ')])
    text = re.sub('  +', ' ', text) # Remove extra spaces
    text = text.strip()
    return text

In [ ]:
def preprocess(text):
    text = lowercase(text) 
    text = remove_unnecessary_char(text)
    text = remove_nonaplhanumeric(text)  
    text = remove_repeated_char(text)
    text = formalize_words(text) 
    text = remove_stopword(text) 
    return text

In [ ]:
def before_preprocess(text):
  text = re.sub(r"\\u00[0-9a-p]{2}","",text)
  text = re.sub(r"[^\u0000-\u007F]"," ",text)
  text = re.sub(r"/p+"," ",text)
  text = re.sub(r"\xa0"," ",text)
  return text

In [ ]:
df['Curhat'] = df['Curhat'].apply(lambda text: before_preprocess(text))
df['Curhat'] = df['Curhat'].apply(lambda text: preprocess(text))

In [ ]:
df.to_csv('clean_data.csv',index=False)